# TRABAJO FINAL INTEGRADOR

### NOMBRE: GUIDO PETTINARI

### TUTOR: VALERIA SOLIANI

### UNIVERSIDAD: INSTITUTO TECNOLOGICO DE BUENOS AIRES (ITBA)

**El Trabajo se dividirá en**:

    1) Análisis Exploratorio Descriptivo
    2) Feature Engineering o Preparación de Datos
    3) Modelos Predictivos: Probaremos diferentes algoritmos predictivos

### Import Libraries

In [1]:
import pandas as pd
from funpymodeling.exploratory import status, profiling_num, freq_tbl, cat_vars, num_vars, corr_pair
from pandas_profiling import ProfileReport
from qgrid import show_grid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import numpy as np
from scipy import stats

### Load Dataframe

In [2]:
data = pd.read_csv('dataset_hr.csv', sep=',')

### Limpieza Columnas que no tomaremos en nuestros modelos

Como definimos que nuestra variable a predecir sea la variable monthly income discretizada, decidimos limpiar las columnas que tengan salario de por medio. Es decir, la siguientes columnas: DailyRate, HourlyRate,MonthlyRate,PercentSalaryHike.\

Por otro lado eliminaremos columnas que sean constantes en toda la tabla (1 solo valor único) y columnas que tienen la misma cantidad de valores únicos que de filas. Es decir, las siguientes columnas: EmployeeCount, Over18, StandardHours. Employee Number lo vamos a dejar ya que nos servirá luego para matchear la predicción con el empleado que corresponde.


In [3]:

data = data.drop(['DailyRate', 'HourlyRate','MonthlyRate','PercentSalaryHike','EmployeeCount',
                  'Over18','StandardHours'], axis=1)

data['MonthlyIncome'].head(5)

0    5993
1    5130
2    2090
3    2909
4    3468
Name: MonthlyIncome, dtype: int64

### Discretización variable a predecir

Para convertir nuestra variable continua "Monthly Income" en categórica deberemos discretizarla. El método para discretizar va a ser por percentiles, es decir por igual frecuencia.\

In [4]:
#Discretize in tens
data['MonthlyIncome_cat']= pd.qcut(data['MonthlyIncome'],q=10)
#Drop Monthly Income
data_base = data.drop(['MonthlyIncome'],axis=1)


In [5]:
status(data_base)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,Age,0,0.0,0,0.000000,43,int64
1,Attrition,0,0.0,0,0.000000,2,object
2,BusinessTravel,0,0.0,0,0.000000,3,object
3,Department,0,0.0,0,0.000000,3,object
4,DistanceFromHome,0,0.0,0,0.000000,29,int64
5,Education,0,0.0,0,0.000000,5,int64
6,EducationField,0,0.0,0,0.000000,6,object
7,EmployeeNumber,0,0.0,0,0.000000,1470,int64
8,EnvironmentSatisfaction,0,0.0,0,0.000000,4,int64
9,Gender,0,0.0,0,0.000000,2,object


### Creación de nuevas variables

Crearemos nuevas variables que puedan enriquecer al modelo. Las nuevas variables serán:

- CompaniesRotation = TotalWorkingYears / NumCompaniesWorked
- PercYearsCurrManager = YearsWithCurrManager / YearsAtCompany
- PercYearsLastPromotion = YearsSinceLastPromotion / YearsAtCompany
- PercYearsCurrentRole = YearsInCurrentRole / YearsAtCompany

In [6]:
data_base['CompaniesRotation'] = data_base['TotalWorkingYears'] / data_base['NumCompaniesWorked']
data_base['PercYearsCurrManager'] = data_base['YearsWithCurrManager'] / data_base['YearsAtCompany']
data_base['PercYearsLastPromotion'] = data_base['YearsSinceLastPromotion'] / data_base['YearsAtCompany']
data_base['PercYearsCurrentRole'] = data_base['YearsInCurrentRole'] / data_base['YearsAtCompany']
data_base['CompaniesRotation'] =  data_base['CompaniesRotation'].replace([np.inf, -np.inf], np.nan)
data_base[['TotalWorkingYears','NumCompaniesWorked','CompaniesRotation']].loc[data_base['CompaniesRotation'] == 40]

,TotalWorkingYears,NumCompaniesWorked,CompaniesRotation
126,40,1,40.0


### Creación multiples dataframes

Como entendemos que todos los algoritmos tienen diferentes supuestos lo que haremos es crear varios dataframes para luego poder correr los modelos. Los dataframes resultantes los llamaremos de la siguiente manera:\

 - data_base = Dataframe sin ningún cambio 
 - data_ohe = Dataframe convirtiendo las variables categoricas con one hot encoding a numéricas. La variable a predecir será convertida por one label encoding 
 - data_ohe_corr = Dataframe quitando variables que esten altamente correlacionadas con la variable a predecir + one hot encoding 
 - data_ohe_out = Dataframe tratando outliers + one hot encoding 
 - data_ohe_out_corr = Dataframe tratando outliers + one hot encoding + quitando variables que esten altamente correlacionadas 
 - data_out = Dataframe tratando outliers 
 - data_corr = Dataframe quitando variables altamente correlacionadas con la variable a predecir
 - data_out_corr = Dataframe sin outliers ni variables correlacionadas



#### Creación data_ohe

In [7]:
### Transformamos con one label encoding la variable a predecir

data_ohe = data_base
le = preprocessing.LabelEncoder()
le.fit(data_ohe['MonthlyIncome_cat'])
data_ohe['MonthlyIncome_cat'] = le.transform(data_ohe['MonthlyIncome_cat']) 


### Transformamos el resto de las variables categoricas en numericas con one hot encoding

data_ohe = pd.get_dummies(data_ohe,drop_first=True)

#### Creación data_ohe_corr

Realizamos nuevamente una matriz de correlacion. Para este ejercicio vamos a quitar la variable a predecir y el employee number. Luego crearemos una función donde tome los titulos que tengan mayor correlación a 0.8 valor absoluto. Luego elimino esas columnas

In [8]:
data_ohe_corr = data_ohe

In [9]:
### Realizamos nuevamente una matriz de correlacion. Para este ejercicio vamos a quitar la variable a predecir y
### el employee number. Luego crearemos una función donde tome los titulos que tengan mayor correlación a 0.8 valor absoluto
### Luego elimino esas columnas
data_ohe_corr_without = data_ohe_corr.drop(['MonthlyIncome_cat','EmployeeNumber'],axis=1)

correlated_features = set()
correlation_matrix = data_ohe_corr_without.corr()

for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
            
data_ohe_corr.drop(labels=correlated_features, axis=1, inplace=True)

#### Data_ohe_corr_out

En este caso para no perder datos vamos a utilizar la tecnica de winsorized, donde pasaremos aquellos outliers por encima del percentil 95 a que tengan el valor correspondiente a dicho percentil y por otro lado los valores inferiores al percentil 5, pasaremos a que tengan valor de dicho percentil

In [10]:
data_ohe_corr_out = data_ohe_corr

In [11]:
#Modificamos los valores que superan el percentil 95 y son menores al percentil 0.05

for col in data_ohe_corr_out[['NumCompaniesWorked','TotalWorkingYears','StockOptionLevel','TrainingTimesLastYear',
                              'YearsSinceLastPromotion','YearsAtCompany','YearsInCurrentRole',
                              'YearsWithCurrManager','CompaniesRotation']].columns: 
    data_ohe_corr_out[col] = stats.mstats.winsorize(data_ohe_corr_out[col], limits=[0.05, 0.10])
     
        

#### Creación data_ohe_out

In [12]:
data_ohe_out = data_ohe

In [13]:

for col in data_ohe_out[['NumCompaniesWorked','TotalWorkingYears','StockOptionLevel','TrainingTimesLastYear',
                              'YearsSinceLastPromotion','YearsAtCompany','YearsInCurrentRole',
                              'YearsWithCurrManager','CompaniesRotation']].columns: 
    data_ohe_out[col] = stats.mstats.winsorize(data_ohe_out[col], limits=[0.05, 0.10])

#### Creación data_out

In [14]:
data_out = data_base

In [15]:
for col in data_out[['NumCompaniesWorked','TotalWorkingYears','StockOptionLevel','TrainingTimesLastYear',
                              'YearsSinceLastPromotion','YearsAtCompany','YearsInCurrentRole',
                              'YearsWithCurrManager','CompaniesRotation']].columns: 
    data_out[col] = stats.mstats.winsorize(data_out[col], limits=[0.05, 0.10])

#### Creación data_corr

In [16]:
data_corr = data_base

In [17]:
### Realizamos nuevamente una matriz de correlacion. Para este ejercicio vamos a quitar la variable a predecir y
### el employee number. Luego crearemos una función donde tome los titulos que tengan mayor correlación a 0.8 valor absoluto
### Luego elimino esas columnas
data_corr_without = data_corr.drop(['MonthlyIncome_cat','EmployeeNumber'],axis=1)

correlated_features = set()
correlation_matrix = data_corr_without.corr()

for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
            
data_corr.drop(labels=correlated_features, axis=1, inplace=True)

#### Creación data_out_corr

In [18]:
data_out_corr = data_corr

data_out_corr

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,MonthlyIncome_cat,CompaniesRotation,PercYearsCurrManager,PercYearsLastPromotion,PercYearsCurrentRole
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,1,2,Female,...,8,1,1,6,0,6,1.000000,0.833333,0.000000,0.666667
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,2,3,Male,...,10,3,3,10,1,5,10.000000,0.700000,0.100000,0.700000
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,4,Male,...,7,3,3,1,0,0,1.166667,NaN,NaN,NaN
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,5,4,Female,...,8,3,3,8,3,2,8.000000,0.000000,0.375000,0.875000
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,7,1,Male,...,6,3,3,2,2,3,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Research & Development,23,2,Medical,2061,3,Male,...,17,3,3,5,0,1,4.250000,0.600000,0.000000,0.400000
1466,39,No,Travel_Rarely,Research & Development,6,1,Medical,2062,4,Male,...,9,5,3,7,1,8,2.250000,1.000000,0.142857,1.000000
1467,27,No,Travel_Rarely,Research & Development,4,3,Life Sciences,2064,2,Male,...,6,1,3,6,0,6,6.000000,0.500000,0.000000,0.333333
1468,49,No,Travel_Frequently,Sales,2,3,Medical,2065,4,Male,...,17,3,2,9,0,5,8.500000,0.888889,0.000000,0.666667


In [19]:
for col in data_out_corr[['NumCompaniesWorked','TotalWorkingYears','StockOptionLevel','TrainingTimesLastYear',
                              'YearsSinceLastPromotion','YearsAtCompany','CompaniesRotation']].columns: 
    data_out_corr[col] = stats.mstats.winsorize(data_out_corr[col], limits=[0.05, 0.10])